In [7]:
import pandas as pd
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [8]:
file = '../Resources/sentiment_analysis_10k.csv' 
twitter_df = pd.read_csv(file,encoding = "ISO-8859-1")

In [9]:
twitter_df

,polarity,id,date,query,user,text,new_date
0,0,2051199119,Fri Jun 05 21:04:50 PDT 2009,NO_QUERY,alicatpurr,"My moonstone pendant fell off today, somewhere...",6/5/09
1,0,2051199378,Fri Jun 05 21:04:56 PDT 2009,NO_QUERY,joshwehe,Watching baseball tonight and saying goodbye t...,6/5/09
2,0,2051200441,Fri Jun 05 21:05:17 PDT 2009,NO_QUERY,qwerkyqook,RIP cute black mac book. Sorry about the olive...,6/5/09
3,0,2051201409,Fri Jun 05 21:05:36 PDT 2009,NO_QUERY,Lizfig3,"@pandafandanga werd. I miss him. Hell man, I m...",6/5/09
4,0,2051201881,Fri Jun 05 21:05:46 PDT 2009,NO_QUERY,sweet_ctstrphe,lost my voice we must only talk through visi...,6/5/09
...,...,...,...,...,...,...,...
19995,4,1690832374,Sun May 03 16:28:55 PDT 2009,NO_QUERY,JaqStone,"@charini Hi, Charini. I see you found my bless...",5/3/09
19996,4,1690832378,Sun May 03 16:28:55 PDT 2009,NO_QUERY,arrogantass,is about to get her blog on! Methinks you will...,5/3/09
19997,4,1690832412,Sun May 03 16:28:55 PDT 2009,NO_QUERY,MzzV,Too much running around today but at least it ...,5/3/09
19998,4,1690832415,Sun May 03 16:28:55 PDT 2009,NO_QUERY,Dutch_Patrick,"@cristina_jacob thanks for the follow, i hope...",5/3/09


In [10]:
# Split up positive and negative tweets into two dataframes
positive_tweets_df = twitter_df.loc[twitter_df["polarity"] ==4]
negative_tweets_df = twitter_df.loc[twitter_df["polarity"] ==0]

train_pos = positive_tweets_df[:8000]
test_pos = positive_tweets_df[8000:]
train_neg = negative_tweets_df[:8000]
test_neg = negative_tweets_df[8000:]

# combine positive and negative labels
train_X = pd.concat([train_pos ,train_neg]) 
test_X = pd.concat([test_pos ,test_neg])

train_X

,polarity,id,date,query,user,text,new_date
5000,4,1687070120,Sun May 03 07:40:31 PDT 2009,NO_QUERY,claireevans86,i love sundays! apartment hunting then lunch w...,5/3/09
5001,4,1687070166,Sun May 03 07:40:31 PDT 2009,NO_QUERY,thiswayup,Finally made it to the outside sunny world and...,5/3/09
5002,4,1687070186,Sun May 03 07:40:31 PDT 2009,NO_QUERY,JillzWorth,@micgeronimo good morning dorkling! Happy Sunday,5/3/09
5003,4,1687070187,Sun May 03 07:40:31 PDT 2009,NO_QUERY,ericdevos,@martjev I repeat: Hints rules We waren wel e...,5/3/09
5004,4,1687070241,Sun May 03 07:40:32 PDT 2009,NO_QUERY,boogaloobrew,@jeneeic3 It's terrible! Falls somewhere betwe...,5/3/09
...,...,...,...,...,...,...,...
8112,0,1559702405,Sun Apr 19 11:49:55 PDT 2009,NO_QUERY,MaggieHackett,@jasonbradford Sorry to hear chickababy didn't...,4/19/09
8113,0,1559702523,Sun Apr 19 11:49:56 PDT 2009,NO_QUERY,vickyjones91,doesnt want school tomorrow ew ew ew,4/19/09
8114,0,1559703098,Sun Apr 19 11:50:03 PDT 2009,NO_QUERY,jahrootz,For some reason when I woke up this morning my...,4/19/09
8115,0,1559703243,Sun Apr 19 11:50:04 PDT 2009,NO_QUERY,rdelizo35,"@Rachealblack110: Sorry to hear that,",4/19/09


In [5]:
train_y = train_X["polarity"]
test_y = train_X["polarity"]

In [6]:
train_X.drop(["polarity"],inplace=True, axis=1)
train_X.drop(["id"],inplace=True, axis=1)
train_X.drop(["date"],inplace=True, axis=1)
train_X.drop(["query"],inplace=True, axis=1)
train_X.drop(["user"],inplace=True, axis=1)
train_X.drop(["new_date"],inplace=True, axis=1)

test_X.drop(["polarity"],inplace=True, axis=1)
test_X.drop(["id"],inplace=True, axis=1)
test_X.drop(["date"],inplace=True, axis=1)
test_X.drop(["query"],inplace=True, axis=1)
test_X.drop(["user"],inplace=True, axis=1)
test_X.drop(["new_date"],inplace=True, axis=1)

In [38]:
print(train_X.shape)
print(test_X.shape)

(16000, 1)
(4000, 1)


In [14]:

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [15]:
train_y

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

# Tokenization

In [11]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
           reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
            if (word not in stopwords_english and  # remove stopwords
                    word not in string.punctuation):  # remove punctuation
                # tweets_clean.append(word)
                stem_word = stemmer.stem(word)  # stemming word
                tweets_clean.append(stem_word)

In [12]:
tweets_clean

NameError: name 'tweets_clean' is not defined

In [17]:
from nltk.corpus import twitter_samples 